## Imports

In [1]:
import pandas as pd
import numpy as np
import os
import folium
import ast
import textwrap
import googlemaps
from folium.plugins import Fullscreen, MiniMap

## Data Read in

In [2]:
# df = pd.read_csv("GroundLeaseCoOps.csv",dtype='str')
df = pd.read_csv("GroundLeaseCoOps - Combined Data For Map.csv")

In [3]:
len(df)

66

In [4]:
df['full_address'] = df['Address'] + ' ' + df['Locale'] + ' New York'

## Get Coordinates

In [5]:
%store -r google_maps_API_Key
gmaps_key = googlemaps.Client(key=google_maps_API_Key)

%store -r map_box_api_key

In [6]:
df.columns

Index(['Address', 'Locale', 'Units', 'full_address'], dtype='object')

In [7]:
# Define the geocode function
def geocode(add):
    g = gmaps_key.geocode(add)
    if g:
        lat = g[0]["geometry"]["location"]["lat"]
        lng = g[0]["geometry"]["location"]["lng"]
        return (lat, lng)
    else:
        return None

# Apply geocoding to the 'geo_address' column and store the results in 'geocoded' column
df['geocoded'] = df['full_address'].apply(geocode)

In [8]:
df['geocoded'] = df['geocoded'].astype(str)
df[['lat', 'lon']] = df['geocoded'].apply(lambda x: (None, None) if x == 'None' else x.strip('()').split(', ', 1)).apply(pd.Series)
df['lat'] = df['lat'].astype(float)
df['lon'] = df['lon'].astype(float)

## Map Work

In [9]:
df.columns

Index(['Address', 'Locale', 'Units', 'full_address', 'geocoded', 'lat', 'lon'], dtype='object')

In [10]:
# columns_to_keep = [
#     'Owner',
#     'LLC Name',
#     'Address',
#     'Number of Units',
#     'Purchase Amount',
#     'Purchase Date', 
#     'Mortgage Amount', 
#     'Lender', 
#     'Still Own?',
#     'In Foreclosure?', 
#     'Other Lawsuits?',
#     'geocoded',
#     'lat',
#     'lon'
# ]

map_df = df

In [11]:
map_df['Units'] = map_df['Units'].str.replace(',','',regex=True ).astype(int)

In [22]:
import pandas as pd
import folium
from folium.plugins import Fullscreen, MiniMap
from collections import defaultdict

# ─── 1. Load your data ─────────────────────────────────────────────────────────
# map_df = pd.read_csv("your_data.csv")
# Or however you get your DataFrame into `map_df`.

# ─── 2. Popup HTML with underline under the address ──────────────────────────
def create_popup(row):
    return f"""
    <div class="popup-content">
      <div style="
           border-bottom:1px solid #000;
           padding-bottom:4px;
           margin-bottom:6px;">
        <h4 style="margin:0">{row['Address']}</h4>
      </div>
      <strong>Number of Units:</strong> {row['Units']}<br>
    </div>
    """

# ─── 3. Build a color map by Locale ───────────────────────────────────────────
unique_locales = map_df['Locale'].unique()
_palette = [
    'red','blue','green','purple','orange','darkred','lightred','beige',
    'darkblue','darkgreen','cadetblue','darkpurple','white','pink',
    'lightblue','lightgreen','gray','black','lightgray'
]
locale_color = {
    loc: _palette[i % len(_palette)]
    for i, loc in enumerate(unique_locales)
}

# ─── 4. Initialize the map (no default tiles) ────────────────────────────────
center_lat = map_df['lat'].mean()
center_lon = map_df['lon'].mean()
m = folium.Map(
    location=[center_lat, center_lon],
    zoom_start=10,
    tiles=None,            # ← kills default OpenStreetMap
    scrollWheelZoom=False
)

# ─── 5. Add your custom Mapbox tiles (hidden from legend) ───────────────────
folium.TileLayer(
    tiles=(
        "https://api.mapbox.com/styles/v1/"
        "mapbox/streets-v11/tiles/256/{z}/{x}/{y}@2x"
        f"?access_token={map_box_api_key}"
    ),
    attr="Mapbox",
    name="Streets (Mapbox)",
    overlay=False,
    control=False,         # ← do NOT include in LayerControl
    show=True,
    min_zoom=1,
    max_zoom=20
).add_to(m)

# ─── 6. CSS tweaks for popups ────────────────────────────────────────────────
m.get_root().html.add_child(folium.Element("""
<style>
  .popup-content { min-width:200px; font-size:13px; }
  .leaflet-popup-content-wrapper { background:#f9f9f9; border-radius:5px; }
  .leaflet-popup-tip { background:#f9f9f9; }
</style>
"""))

# ─── 7. Fixed-position title ─────────────────────────────────────────────────
title_html = """
<div style="
  position: fixed;
  top: 10px;
  left: 50%;
  transform: translateX(-50%);
  z-index: 1001;
  background: white;
  padding: 5px 10px;
  border-radius: 4px;
  border: 1px solid #ccc;
">
  <h3 style="margin:0; font-family:Arial, sans-serif;">
    Ground Lease Co-Ops
  </h3>
</div>
"""
m.get_root().html.add_child(folium.Element(title_html))

# ─── 8. Fullscreen & minimap controls ────────────────────────────────────────
Fullscreen().add_to(m)
MiniMap(toggle_display=True).add_to(m)

# ─── 9. Plot each site as a CircleMarker, grouped by Locale ─────────────────
groups = defaultdict(folium.FeatureGroup)
for _, row in map_df.iterrows():
    lat, lon = row['lat'], row['lon']
    popup = folium.Popup(create_popup(row), max_width=250)
    radius = max(4, min(row['Units'] / 50, 20))
    circle = folium.CircleMarker(
        location=(lat, lon),
        radius=radius,
        color=locale_color[row['Locale']],
        fill=True, fill_opacity=0.7,
        popup=popup
    )
    groups[row['Locale']].add_child(circle)

for locale, fg in groups.items():
    fg.layer_name = locale
    m.add_child(fg)

folium.LayerControl(collapsed=False).add_to(m)

# ─── 10. Build & inject the “snap-to” dropdown (below the title) ────────────
map_js_var = m.get_name()  # e.g. "map_123abc"
options = "\n".join(
    f'<option value="{r.lat},{r.lon}">{r.Address}</option>'
    for _, r in map_df.iterrows()
)
dropdown_html = f"""
<div style="
  position: fixed;
  top: 60px;           /* ↓ pushed down from the title */
  left: 50px;
  z-index: 1001;
  background: white;
  padding: 5px;
  border: 1px solid #ccc;
  border-radius: 4px;
">
  <select id="addressSelector">
    <option value="">🔍 Jump to address…</option>
    {options}
  </select>
</div>
<script>
  document.addEventListener('DOMContentLoaded', function() {{
    var map = window["{map_js_var}"];
    document.getElementById('addressSelector')
      .addEventListener('change', function() {{
        var parts = this.value.split(',');
        if (parts.length === 2) {{
          map.setView([ parseFloat(parts[0]), parseFloat(parts[1]) ], 16);
        }}
      }});
  }});
</script>
"""
m.get_root().html.add_child(folium.Element(dropdown_html))

# ─── 11. Render or save ──────────────────────────────────────────────────────
# In Jupyter, just display `m`:
m

# Or, if running as a standalone script:
# m.save("interactive_buildings_map.html")


In [23]:
m.save('index.html')

In [24]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/GroundLeaseCoOps
